# Process images and create labels for YOLO

In [2]:
from azure.storage.blob import BlobServiceClient
from pydicom import dcmread
from io import BytesIO
import numpy as np
import cv2
import pylidc as pl

try:
    with open('/home/andrew/ITRI-LungCancer/keys.txt', 'r') as file:
        data = file.read().splitlines()
        account_name    = data[0]
        account_key     = data[1]
        container_name  = data[2]
    
    blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)
    container_client = blob_service_client.get_container_client(container_name)
    blob_name_list = container_client.list_blob_names()
except Exception as ex:
    print('Exception:')
    print(ex)

# Organize file system

In [1]:
# Clean folders
# !rm -rf /home/andrew/ITRI-LungCancer/dataset/
# !rm -rf /home/andrew/ITRI-LungCancer/runs

# Recreate dataset structure
!mkdir -p /home/andrew/ITRI-LungCancer/dataset_big/images/{train,val,test}
!mkdir -p /home/andrew/ITRI-LungCancer/dataset_big/labels/{train,val,test}

# Helper Functions for Creating Dataset

In [3]:
def window_img(img, window_center, window_width):
    win_min = window_center - window_width / 2.0
    win_max = window_center + window_width / 2.0
    img = np.clip(img, win_min, win_max)
    img = (img - win_min) / (win_max - win_min)
    img = np.uint8(img * 255)
    return img

def rescale_img(ds, img):
    if 'RescaleIntercept' in ds and 'RescaleSlope' in ds:
        img = img * ds.RescaleSlope + ds.RescaleIntercept
    return img

def save_if_annotated(scan, ds, slice_location, blob_name, data_string):
    scan_name = blob_name.split('/')[0]
    slice_num = blob_name.split('/')[3].split('-')[1]
    
    for ann_count, ann in enumerate(scan.annotations):
        for contour in ann.contours:
            if abs(contour.image_z_position - slice_location) < scan.slice_spacing and ann.boolean_mask().sum() > 400:
                bbox = ann.bbox()
                bbox_x_center = (bbox[1].start + bbox[1].stop) / ds.Columns / 2
                bbox_y_center = (bbox[0].start + bbox[0].stop) / ds.Rows / 2
                bbox_width = (bbox[1].stop - bbox[1].start)/ds.Columns
                bbox_height = (bbox[0].stop - bbox[0].start)/ds.Rows
                
                image = rescale_img(ds, ds.pixel_array)
                image = window_img(image, -300, 2000)

                filename = f"{scan_name}_{slice_num}_{ann_count}"
                
                image_path = f'/home/andrew/ITRI-LungCancer/dataset_big/images/{data_string}/{filename}.png'
                cv2.imwrite(image_path, image)
                
                label_path = f'/home/andrew/ITRI-LungCancer/dataset_big/labels/{data_string}/{filename}.txt'
                label_txt = f"0 {bbox_x_center} {bbox_y_center} {bbox_width} {bbox_height}"
                with open(label_path, 'w') as file:
                    file.write(label_txt)
                return True
    return False

def create_dataset(count, data_string):
    while(count > 0):
        blob_name = next(blob_name_list)
        
        blob_client = container_client.get_blob_client(blob_name)
        blob_data = blob_client.download_blob().readall()
        blob_stream = BytesIO(blob_data)
        ds = dcmread(blob_stream)
        
        scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == ds.PatientID).first()
        slice_location = ds.ImagePositionPatient[2]
        
        if save_if_annotated(scan, ds, slice_location, blob_name, data_string):
            count -= 1
            if count > 0:
                print(f"{data_string}: {count}    ", end='\r', flush=True)
    print(f"{data_string} done!")

# Create Datasets

In [4]:
create_dataset(8000, "train")
create_dataset(2000, "val")
create_dataset(200, "test")